In [1]:
import numpy as np
from datetime import datetime


def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


def sigmoid(z):
    
    return 1 / (1+np.exp(-z))

In [2]:
class SimpleClassificationTest:
    
    # constructor
    def __init__(self, xdata, tdata, learning_rate, iteration_count):
            
        # 가중치 W 형상을 자동으로 구하기 위해 입력데이터가 vector 인지,
        # 아니면 matrix 인지 체크 후, 
        # self.xdata 는 무조건 matrix 로 만들어 주면 코드 일관성이 있음
        
        if xdata.ndim == 1:    # vector
            self.xdata = xdata.reshape(len(xdata), 1)
            self.tdata = xdata.reshape(len(tdata), 1)
            
        elif xdata.ndim == 2:  # matrix
            self.xdata = xdata
            self.tdata = tdata
        
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W = np.random.rand(self.xdata.shape[1], 1) 
        self.b = np.random.rand(1)
        
        print("SimpleClassificationTest Object is created")
        
        
    # obtain current W and current b
    def getW_b(self):
        
        return self.W, self.b
    
    
    # loss function
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
        
    
    # display current error value
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
    
    
    # predict method
    # 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
    # 입력변수 x : numpy type
    def predict(self, test_data):
    
        z = np.dot(test_data, self.W) + self.b
        y = sigmoid(z)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    
    # train method
    def train(self):
    
        f = lambda x : self.loss_func()

        print("Initial error value = ", self.error_val() )

        start_time = datetime.now()
        
        for step in  range(self.iteration_count):  
    
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % 2000 == 0):
                print("step = ", step, "error value = ", self.error_val() )
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [3]:
# 입력데이터 / 정답데이터 세팅

x_data = np.array([2, 4, 6, 8, 10, 12, 14, 16, 18, 20]).reshape(10,1)   
t_data = np.array([0, 0, 0, 0,  0,  0,  1,  1,  1,  1]).reshape(10,1)

print("x_data.shape = ", x_data.shape, ", t_data.shape = ", t_data.shape)

x_data.shape =  (10, 1) , t_data.shape =  (10, 1)


### learning_rate = 1e-2,  반복횟수 100,000번 수행하는 obj1

In [4]:
obj1 = SimpleClassificationTest(x_data, t_data, 1e-2, 100001)

obj1.train()

SimpleClassificationTest Object is created
Initial error value =  17.76318343849415
step =  0 error value =  8.335790549755924
step =  2000 error value =  1.2331329012892043
step =  4000 error value =  0.9287804993318488
step =  6000 error value =  0.781666042769222
step =  8000 error value =  0.6882020647578967
step =  10000 error value =  0.6210608418729111
step =  12000 error value =  0.5693277774481802
step =  14000 error value =  0.5276249816031385
step =  16000 error value =  0.4929340184783088
step =  18000 error value =  0.46340207335727374
step =  20000 error value =  0.43781473145318306
step =  22000 error value =  0.4153350247484534
step =  24000 error value =  0.3953627592431277
step =  26000 error value =  0.37745346101612015
step =  28000 error value =  0.36126910652333516
step =  30000 error value =  0.34654682535466563
step =  32000 error value =  0.3330782722402807
step =  34000 error value =  0.32069559750374454
step =  36000 error value =  0.30926164357119756
step = 

In [5]:
test_data = np.array([13.0])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.52082833] 1


In [6]:
test_data = np.array([11.0])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.00611] 0


In [7]:
test_data = np.array([31.0])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[1.] 1


In [8]:
test_data = np.array([3.0])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[6.29074654e-12] 0


In [9]:
test_data = np.array([3.5])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[2.29391463e-11] 0
